# HMM - Viterbi decode

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Define HMM class

In [10]:
class HMM(object):
    
    def __init__(self, initial_prob, trans_prob, obs_prob):
        self.N = np.size(initial_prob)
        self.initial_prob = initial_prob
        self.trans_prob = trans_prob
        self.emission = tf.constant(obs_prob)
        assert self.initial_prob.shape == (self.N, 1)
        assert self.trans_prob.shape == (self.N, self.N)
        assert obs_prob.shape[0] == self.N
        # Placeholders for forward algorithm
        self.obs_idx = tf.placeholder(tf.int32)
        self.viterbi = tf.placeholder(tf.float64)
    
    def get_emission(self, obs_idx):
        """ Helper function for getting slice from emission matrix
            We get observation and return the potentials of a given hidden variables.
        """
        slice_location = [0, obs_idx]
        num_rows = tf.shape(self.emission)[0]
        slice_shape = [num_rows, 1]
        return tf.slice(self.emission, slice_location, slice_shape)
    
    def forward_init_op(self):
        obs_prob = self.get_emission(self.obs_idx)
        # Multiplication element-wise
        return tf.multiply(self.initial_prob, obs_prob)
    
    def decode_op(self):
        transitions = tf.matmul(self.viterbi, tf.transpose(self.get_emission(self.obs_idx)))
        weighted_transistions = transitions * self.trans_prob
        viterbi = tf.reduce_max(weighted_transistions, 0)
        return tf.reshape(viterbi, tf.shape(self.viterbi))
        
    def backpt_op(self):
        back_transitions = tf.matmul(self.viterbi, np.ones((1, self.N)))
        weighted_back_transitions = back_transitions * self.trans_prob
        return tf.argmax(weighted_back_transitions, 0)

## Viterbi algorithm

In [16]:
def viterbi_decode(sess, hmm, observations):
    viterbi = sess.run(hmm.forward_init_op(), feed_dict={hmm.obs_idx: observations[0]})
    backpts = np.ones((hmm.N, len(observations)), 'int32') * -1
    for t in range(1, len(observations)):
        viterbi, backpt = sess.run([hmm.decode_op(), hmm.backpt_op()], 
                                   feed_dict={hmm.obs_idx: observations[t], hmm.viterbi: viterbi})
        backpts[:, t] = backpt
    tokens = [viterbi[:, -1].argmax()]
    for i in range(len(observations) -1, 0, -1):
        tokens.append(backpts[tokens[-1], i])
    return tokens[::-1]

## Example

Let's run it on the following example (wether is a hidden variable):

```json
states = ('Rainy', 'Sunny')
 
observations = ('walk', 'shop', 'clean')
 
start_probability = {'Rainy': 0.6, 'Sunny': 0.4}
 
transition_probability = {
   'Rainy' : {'Rainy': 0.7, 'Sunny': 0.3},
   'Sunny' : {'Rainy': 0.4, 'Sunny': 0.6},
   }
 
emission_probability = {
   'Rainy' : {'walk': 0.1, 'shop': 0.4, 'clean': 0.5},
   'Sunny' : {'walk': 0.6, 'shop': 0.3, 'clean': 0.1},
   }
```   

In [19]:
initial_prob = np.array([[0.6], 
                         [0.4]])        
trans_prob = np.array([[0.7, 0.3],
                       [0.4, 0.6]])
obs_prob = np.array([[0.1, 0.4, 0.5],
                     [0.6, 0.3, 0.1]])

hmm = HMM(initial_prob, trans_prob, obs_prob)

observations = [2, 0, 1, 0, 1]
observations2 = [2, 2, 2, 2, 2]
with tf.Session() as sess:
    prob = viterbi_decode(sess, hmm, observations)
    print('Most probable weather {} is {}'.format(observations, prob))
    prob = viterbi_decode(sess, hmm, observations2)
    print('Most probable weather {} is {}'.format(observations2, prob))    

Most probable weather [2, 0, 1, 0, 1] is [0, 1, 1, 1, 1]
Most probable weather [2, 2, 2, 2, 2] is [0, 0, 0, 0, 0]
